Web scraping names and details of stocks with max Gain and Loss using selenium webdriver

We are scrapping the names of top ten and bottom 10 stocks from BSE website, and later the names of each stock is then fed to the twitter scraper.

twitter scraper uses the query to scrape tweets related to the stock names, the time frame taken is past 7 months.

We are saving the scraped tweets along with the information on username, time and a flag to identify the stock status

stock staus 0 is a loser stock and 1 denotes its a gainer stock

In [2]:
#Libraries for webscraping

import selenium
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time

In [3]:
#setting up browser config for web driver

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
driverPATH =  r'/home/alby/Downloads/chromedriver'
browserBIN = '/usr/bin/vivaldi'

In [4]:
#setting up browser config for web driver
options = webdriver.ChromeOptions()
options.binary_location = browserBIN

In [5]:
##setting up browser config for web driver
driver = webdriver.Chrome(driverPATH, options=options)

/tmp/ipykernel_33648/2275245732.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driverPATH, options=options)


In [49]:
#accessing url for the site with stock info

url="https://www.bseindia.com/markets.html"
driver.get(url)
time.sleep(5)

In [50]:
#accessing the urls of top 10 gainers and losers

gain_table=driver.find_elements(By.XPATH,"//tbody[@class='ng-scope'and @ng-repeat='G in GainersAndLoosers.Table track by G.scrip_cd']")
links_gain=[]
for j in gain_table:
    time.sleep(2)

    link_tag=j.find_element(By.TAG_NAME,"a")
    links_gain.append(link_tag.get_attribute('href'))

#clicking to access the losers column


links_loss=[]

time.sleep(5)

loss=driver.find_element(By.XPATH,'//*[@id="idLosers"]')
loss.click()

time.sleep(2)

#accessing the table

loss_table=driver.find_elements(By.XPATH,"//tbody[@class='ng-scope'and @ng-repeat='L in GainersAndLoosers1.Table track by L.scrip_cd']")


for l in loss_table:

    tag=l.find_element(By.TAG_NAME,"a")
    links_loss.append(tag.get_attribute('href'))





The url for each stocks is scraped

In [52]:
#getting names of top stocks in gain and loss
gain_stocks=[]
loss_stocks=[]
for name in links_gain:
    gain_stocks.append(name.split("/")[4].replace("-"," ").replace("ltd"," ").strip())

#loss stocks
for names in links_loss:
    loss_stocks.append(names.split("/")[4].replace("-"," ").replace("ltd"," ").strip())




The top 10 stocks are saved in gain stocks and the bottom 10 loss_stocks

In [63]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
###Sns scraping-Twitter for tweets on stocks
tweets = []



for gainer in gain_stocks:

    query = str(gainer)  + " lang:en until:2022-07-12 since:2022-06-01"
    
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        tweets.append([tweet.date, tweet.username, tweet.content,gainer,1])

/tmp/ipykernel_33648/3767384733.py:13: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content,gainer,1])


In [66]:
#Scraping tweets for bottom ten stocks

ltweets=[]


for loser in loss_stocks:

    query = str(loser)+ " lang:en until:2022-07-12 since:2022-06-01"

    for twee in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
            ltweets.append([twee.date, twee.username, twee.content,loser,0])




/tmp/ipykernel_33648/1943940824.py:14: FutureWarning: username is deprecated, use user.username instead
  ltweets.append([twee.date, twee.username, twee.content,loser,0])


In [67]:
#checking total number of tweets
len(tweets)

18212

In [68]:
#creating the dataframe for scraped data

df1 = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet','company','Stock_status'])

df2 = pd.DataFrame(ltweets, columns=['Date', 'User', 'Tweet','company','Stock_status'])


In [74]:
#merging the dataframe
df=pd.concat([df1,df2])


In [76]:
#saving as csv
df.to_csv('Stocks_twitter.csv')
